# Updated WD

In [1]:
from ibm_cloud_sdk_core.authenticators import BearerTokenAuthenticator
from ibm_watson import DiscoveryV2
from ibm_watson.discovery_v2 import QueryLargePassages
import traceback
import os
import json
from dotenv import load_dotenv
from utils.logger import logger
import itertools
from langchain_ibm import WatsonxLLM
from langchain_ibm import ChatWatsonx
import requests
from utils.llm_utils import call_with_timeout
from langchain.prompts import PromptTemplate

import httpx
import traceback

from ibm_cloud_sdk_core.api_exception import ApiException

In [2]:

try:
    # STEP-1: Read the env variables
    WD_BEARER_TOKEN = os.environ["WD_BEARER_TOKEN"]
    WD_VERSION = os.environ["WD_VERSION"]
    WD_URL = os.environ["WD_URL"]
    WD_RAN_PROJECT_ID = os.environ["WD_RAN_PROJECT_ID"]
    WD_PASSAGE_CHARACTERS_LIMIT = int(os.environ["WD_PASSAGE_CHARACTERS_LIMIT"])
    # WATSONX_MODEL_ID_LLAMA = os.environ["WATSONX_MODEL_ID_LLAMA"]
    # LLM_LLAMA_PARAMS =  os.environ["LLM_LLAMA_PARAMS"]
except Exception as e:
    print(e)
    print("Loading Environmment Variables from local .env file")
    load_dotenv()
    WD_BEARER_TOKEN = os.environ["WD_BEARER_TOKEN"]
    WD_VERSION = os.environ["WD_VERSION"]
    WD_URL = os.environ["WD_URL"]
    WD_RAN_PROJECT_ID = os.environ["WD_RAN_PROJECT_ID"]
    WD_PASSAGE_CHARACTERS_LIMIT = int(os.environ["WD_PASSAGE_CHARACTERS_LIMIT"])
    # WATSONX_MODEL_ID_LLAMA = os.environ["WATSONX_MODEL_ID_LLAMA"]
    # LLM_LLAMA_PARAMS =  os.environ["LLM_LLAMA_PARAMS"]


try:
    # Fetch environment variables
    WATSONX_URL = os.environ["WATSONX_URL"]
    WATSONX_APIKEY = os.environ["WATSONX_API_KEY"]
    WATSONX_PROJECT_ID = os.environ["WATSONX_PROJECT_ID"]
    WATSONX_MODEL_ID_LLAMA = os.environ["WATSONX_MODEL_ID_LLAMA"]

    LLM_LLAMA_PARAMS = os.environ["LLM_LLAMA_PARAMS"] 
    # print(LLM_LLAMA_PARAMS)
    TIMEOUT_DURATION_LLM_ANALYZER = int(os.environ["TIMEOUT_DURATION_LLM_ANALYZER"])
    TIMEOUT_DURATION_LLM = int(os.environ["TIMEOUT_DURATION_LLM"])
    LLM_LLAMA_PARAMS_CHAT = os.environ["LLM_LLAMA_PARAMS_CHAT"]
    WD_COLLECTION_ID = os.environ["WD_COLLECTION_ID"]
except Exception as e:
    print(e)
    print("Loading Environmment Variables from local .env file")
    load_dotenv()
    WATSONX_URL = os.environ["WATSONX_URL"]
    WATSONX_APIKEY = os.environ["WATSONX_API_KEY"]
    WATSONX_PROJECT_ID = os.environ["WATSONX_PROJECT_ID"]
    WATSONX_MODEL_ID_LLAMA = os.environ["WATSONX_MODEL_ID_LLAMA"]

    LLM_LLAMA_PARAMS = os.environ["LLM_LLAMA_PARAMS"]
    # print(LLM_LLAMA_PARAMS)
    TIMEOUT_DURATION_LLM_ANALYZER = int(os.environ["TIMEOUT_DURATION_LLM_ANALYZER"])
    TIMEOUT_DURATION_LLM = int(os.environ["TIMEOUT_DURATION_LLM"])
    LLM_LLAMA_PARAMS_CHAT = os.environ["LLM_LLAMA_PARAMS_CHAT"]
    WD_COLLECTION_ID = os.environ["WD_COLLECTION_ID"]


llm_llama = WatsonxLLM(
    model_id=WATSONX_MODEL_ID_LLAMA,
    params=json.loads(LLM_LLAMA_PARAMS),
    project_id=WATSONX_PROJECT_ID,
    url=WATSONX_URL,
    apikey=WATSONX_APIKEY
)

chatmodel = ChatWatsonx(
    model_id=WATSONX_MODEL_ID_LLAMA,
    project_id=WATSONX_PROJECT_ID,
    url=WATSONX_URL,
    apikey=WATSONX_APIKEY,
    params=json.loads(LLM_LLAMA_PARAMS_CHAT),
    streaming=False,
)

 # Define variables for WD_URL_QUERY
# url = "https://cpd-cpd-inst-operands.apps.watsonx.cpni-wl-dw-d.aws.dishcloud.io/discovery/cpd-inst-operands-wd/instances/1730206863041206/api/v2/projects/826cc215-0482-454f-b649-217416edb7e3/query?version=2022-08-01"
WD_URL_QUERY = f"{WD_URL}/v2/projects/{WD_RAN_PROJECT_ID}/query?version={WD_VERSION}"
# WD_URL_QUERY = f"{WD_URL}/v2/projects/{WD_RAN_PROJECT_ID}/collections/{WD_COLLECTION_ID}/query?version={WD_VERSION}"
WD_URL_QUERY

'https://cpd-cpd-inst-operands.apps.watsonx.cpni-wl-dw-d.aws.dishcloud.io/discovery/cpd-inst-operands-wd/instances/1730206863041206/api/v2/projects/826cc215-0482-454f-b649-217416edb7e3/query?version=2022-08-01'

In [3]:
async def analyzer(state):

    """

    """
    prompt_analyzer='As a NOC expert, answer in detail.\nDocument: {passages}\nInput: {user_question}\nOutput:'
    try:
        logger.log(f'user_question:{state.get("user_query")} - RAN-Analyzer : START')
        prompt_sql_text = PromptTemplate(
            input_variables=["user_question", "passages", "classifier"],
            #template=ANALYZER_PROMPT_TEMPLATE_LLAMA,
            template=prompt_analyzer,
        )
        # Use context dictionary directly
        result_context = {
            "user_question": state.get('user_question'),
            "passages": state.get('passages'),
            "classifier": state.get('classifier')
        }
        # Print and invoke LLM with formatted template
        final_template = prompt_sql_text.format(**result_context)

        async with call_with_timeout(
                chatmodel.ainvoke(final_template), "analyzer",timeout = TIMEOUT_DURATION_LLM_ANALYZER
            ) as summary:
            logger.log(f'user_question:{state.get("user_query")} - RAN-Analyzer : END')
            return summary.content

    except Exception as e:
        traceback.print_exc()
        logger.log(
               f'ran-analyzer - Error in analyzer  :: {traceback.print_exc()}'
        )
        return None

In [6]:
# # connect to WD
# async def wd_connect():
#     authenticator = BearerTokenAuthenticator(WD_BEARER_TOKEN)
#     discovery = DiscoveryV2(
#         version=WD_VERSION,
#         authenticator=authenticator
#     )
#     discovery.set_service_url(WD_URL)
#     discovery.set_disable_ssl_verification(True)
#     return discovery


In [7]:
# # connect to WD
# async def wd_connect():
#     authenticator = BearerTokenAuthenticator(WD_BEARER_TOKEN)
#     discovery = DiscoveryV2(
#         version=WD_VERSION, # "2022-08-01"
#         authenticator=authenticator
#     )
#     discovery.set_service_url(WD_URL)
#     discovery.set_disable_ssl_verification(True)
#     return discovery


# discovery = await wd_connect()
# passages_settings = QueryLargePassages(
#     enabled=True,
#     per_document=True,
#     characters=5000,
#     fields=["text"],
#     find_answers=True
# )
# user_query = "what is the Dish GPL Value for the parameter cbPreamblePerSSB ?"
# discovery_json = discovery.query(
#         project_id= WD_RAN_PROJECT_ID,
#         # query=user_query,
#         natural_language_query=user_query,
#         passages=passages_settings,
#         count=3,
#     ).get_result()

In [8]:
# data_json_string=json.dumps(discovery_json)
# data = json.loads(data_json_string)
# with open('python_response.json', 'w') as json_file:
#     json.dump(data, json_file, indent=4)

In [9]:
# resp_ = await wd_query_format_response(discovery_json)
# # resp_
# resp_['user_question'] = user_query
# summary = await analyzer(resp_)
# summary

In [10]:
# for item in discovery_json['results']:
#     print(item['result_metadata'])

NameError: name 'discovery_json' is not defined

In [ ]:
# # Step 3: Query entities after processing is complete
# async def wd_query_entities_old(user_query, filter_params=None):
#     try: 
#         logger.log(f'ran-wd_query_entities :: START')
#         # Obect creation
#         discovery = await wd_connect()
#         passages_settings = QueryLargePassages(
#             enabled=True,
#             per_document=True,
#             # max_per_document=1,
#             # count=3,  # Max number of passages per document
#             characters=1500,  # Limit each passage to 500 characters
#             fields=["text"],  # Extract passages from these fields
#             find_answers=True
#         )
#         # filter_n = {"filter":{"enriched_text.entities.text:vendor is samsung"}}
#         # # filter_n = 'enriched_text.entities.text:"vendor is samsung" OR enriched_text.keywords.text:"vendor is samsung"'
#         # # filter_n = 'enriched_text.entities.text:"vendor is mavenir"'


#         discovery_json = discovery.query(
#                 project_id= WD_RAN_PROJECT_ID,
#                 # filter=filter_params,
#                 # filter='enriched_text.entities.text:"vendor is dish"',
#                 query=user_query,
#                 # natural_language_query=user_query,
#                 passages=passages_settings,
#                 count=3,
#             ).get_result()
#         logger.log(f'ran-wd_query_entities :: END')
#         return discovery_json
#     except:
#         print(traceback.format_exc())
#         print("Error in watson discovery query entities")
#         return {}

In [11]:
# # Step 3: Query entities after processing is complete
# async def wd_query_entities(user_query, filter_params=None):
#     try: 
#         logger.log(f'ran-wd_query_entities :: START')
#         ####
#         headers = {
#             "Content-Type": "application/json",
#             "Authorization": f"Bearer {WD_BEARER_TOKEN}",
#         }
#         payload = {
#             "count": 3,
#             "return": ["title", "metadata"],
#             "passages": {
#                 "enabled": True,
#                 "characters": WD_PASSAGE_CHARACTERS_LIMIT,
#                 "per_document": True,
#                 "find_answers": True,
#                 "fields": ["text"]
#             },
#             # "filter": filter_vendor,
#             "natural_language_query":user_query,
#             "table_results": {"enabled": False}
#         }
#         # Send the POST request
#         response = requests.post(WD_URL_QUERY, headers=headers, json=payload, verify=False)

#         # Check response status and content
#         if response.status_code == 200:
#             logger.log(f'ran-wd_query_entities :: END')
#             return response.json()
#         else:
#             logger.log(f'"Error occurred, please check the server"')
#             logger.log(f'ran-wd_query_entities :: END')
#             return {}
#     except:
#         print(traceback.format_exc())
#         print("Error in watson discovery query entities")
#         return {}

In [4]:


# Step 3: Query entities after processing is complete
async def wd_query_entities(user_query, filter_params=None):
    try:
        logger.log(f'ran-wd_query_entities :: START')
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {WD_BEARER_TOKEN}",
        }
        payload = {
            "count": 3,
            "return": ["title", "metadata"],
            "passages": {
                "enabled": True,
                "characters": 5000,
                "per_document": True,
                "find_answers": True,
                "fields": ["text"],
            },
            'collection_ids' : [WD_COLLECTION_ID],
            "natural_language_query": user_query,
            "table_results": {"enabled": False},
        }
        
        async with httpx.AsyncClient(verify=False) as client:
            response = await client.post(WD_URL_QUERY, headers=headers, json=payload)

        if response.status_code == 200:
            logger.log(f'ran-wd_query_entities :: END')
            return response.json()
        else:
            logger.log(f'Error occurred, please check the server: {response.status_code}, {response.text}')
            logger.log(f'ran-wd_query_entities :: END')
            return {}
    except Exception as e:
        logger.log(f"Exception in wd_query_entities: {str(e)}")
        print(traceback.format_exc())
        return {}

In [5]:
async def wd_query_format_response(data):
    try:
        # Preprocess the results
        logger.log(f'ran-wd_query_format_response :: START')
        # data_json_string=json.dumps(watsonx_output)
        # data = json.loads(data_json_string)
        # with open('output_1.json', 'w') as json_file:
        #     json.dump(data, json_file, indent=4)
        result_dict = {
                "s3_url": [
                    doc.get('metadata', {}).get('s3_url', 'NA') for doc in data['results']
                ],
                "confidence_score": [doc.get('result_metadata', {}).get('confidence', 0) for doc in data['results']],
                "passages" : ' '.join([
                    ' '.join([
                        s.get('passage_text', '')
                        .replace("<em>", " ")
                        .replace("</em>", " ")
                        .replace("\n", " ")
                        .replace("\t", " ") or ''
                        for s in doc.get('document_passages', [])
                    ]) if isinstance(doc.get('document_passages', []), list) and doc.get('document_passages', []) else ''
                    for doc in data['results']
                ])
            }
        
        logger.log(f'ran-wd_query_format_response :: END - {len(result_dict.get("passages"))}')
        return result_dict
    except Exception as e:
        return None

In [14]:

# async def main_wd_query_and_process(user_query, classifier):
#     ## prepare vendor and release
#     logger.log(f'ran-main_wd_query_and_process :: END')
#     try:
#         vendors = [val for val in classifier.get("vendor",[]) if val.lower() not in ["not_defined", 'other', 'na']]
#         release = [str(val).lower() for val in classifier.get("release",[]) if str(val).lower() != "not_defined"]
#         response_list = []
#         if vendors:
#             for value in vendors:
#                 if release:
#                     filter_str = f"vendor is {value} and release is {', '.join(map(str, release))}"
#                 else:
#                     filter_str = f"vendor is {value}"
#                 ##
#                 print(filter_str)
#                 # Query WD here for each filter set
#                 if filter_str:
#                     resp = await wd_query_entities(user_query, filter_params = filter_str)
#                 else:
#                     resp = None

#                 if resp:
#                     # Call preprocessing part here
#                     response_formated = await wd_query_format_response(resp)
#                     response_list.append(response_formated)
#                 else:
#                     response_formated = None

#         else:
#             print("No filter found, querying wd without any filters")
#             resp = await wd_query_entities(user_query)
#             if resp:
#                 response_formated = await wd_query_format_response(resp)
#                 response_list.append(response_formated)
                
#         if response_list:
#             passages = [item.get("passages", "NA") for item in response_list]
#             s3_urls_ = [item.get("s3_url", "NA")for item in response_list]
#             s3_urls = list(set(itertools.chain(*s3_urls_)))
#             logger.log(f'ran-main_wd_query_and_process :: END')
#             return {"s3_urls":s3_urls, "passages":passages}
#         else:
#             logger.log(f'ran-main_wd_query_and_process :: END')
#             return None
#     except Exception as e:
#         print("e")
#         logger.log(f'ran-main_wd_query_and_process :: ERROR')
#         return None

In [15]:
# user_query =  "what is the Dish GPL Value for the parameter  cbPreamblePerSSB ?"
# data = await wd_query_entities(user_query)

In [16]:
# data['results']

In [17]:

# async def main_wd_query_and_process(user_query):
#     ## prepare vendor and release
#     logger.log(f'ran-main_wd_query_and_process :: END')
#     try:
#         resp = await wd_query_entities(user_query) # don't pass filter for now
#         if resp:
#             response_formated = await wd_query_format_response(resp)
#             logger.log(f'ran-main_wd_query_and_process :: END')
#             return response_formated
#         else:
#             logger.log(f'ran-main_wd_query_and_process :: END')
#             return None
#     except Exception as e:
#         print("e")
#         logger.log(f'ran-main_wd_query_and_process :: ERROR')
#         return None

In [6]:
async def main_wd_query_and_process(user_query):
    ## prepare vendor and release
    logger.log(f'ran-main_wd_query_and_process :: END')
    try:
        resp = await wd_query_entities(user_query) # don't pass filter for now
        if resp:
            response_formated = await wd_query_format_response(resp)
            logger.log(f'ran-main_wd_query_and_process :: END')
            return response_formated
        else:
            logger.log(f'ran-main_wd_query_and_process :: END')
            return None
    except ApiException as e:
            logger.log(f'ran-main_wd_query_and_process :: ApiException occurred :: Attempttng again :: {str(e)}')
            resp = await wd_query_entities(user_query) # don't pass filter for now
            if resp:
                response_formated = await wd_query_format_response(resp)
                logger.log(f'ran-main_wd_query_and_process :: END')
                return response_formated
            else:
                logger.log(f'ran-main_wd_query_and_process :: END')
                return None                 
    except Exception as e:
        print("e")
        logger.log(f'ran-main_wd_query_and_process :: ERROR')
        return None

In [11]:

    
# user_query = "How does Codec Adaption works in Mavenir? any difference with Samsung?"
# user_question = "what is the Dish GPL Value for the parameter  cbPreamblePerSSB ?"
# user_question = 'Describe the a2-criteria-info of connected mobility for frequency band n70 - AWS 4/1900'
# user_question = 'what are the Cell Resel Intra and Inter Freq Info for serving band n70?'
# user_question = 'what are the Internal Mavenir Fixes avalable in Mavenir 5G SA FDD 5232 Release Note ?'
# user_question = "5G vDU (uADPF) Parameter Description for SVR23B NR P2"
user_question = "What is the default value Dish/Boost have in GPL for scellDeactInterval parameter?"
# classifier = {'query_type':['ran_docs'], 'query_category':['feature'], 'vendor':['mavenir','samsung'], 'release': ['NOT_DEFINED']}
# classifier = {'query_type': ['ran_docs'], 'query_category': ['gpl'], 'vendor': ['dish'], 'release': ['NOT_DEFINED']}
# classifier = {'query_type':['ran_docs'], 'query_category':['feature'], 'vendor':['NOT_DEFINED'], 'release': ['NOT_DEFINED']}

resp = await main_wd_query_and_process(user_question)

2025-01-15 13:06:43 - INFO - ran-main_wd_query_and_process :: END
2025-01-15 13:06:43 - INFO - ran-wd_query_entities :: START
2025-01-15 13:06:46 - INFO - ran-wd_query_entities :: END
2025-01-15 13:06:46 - INFO - ran-wd_query_format_response :: START
2025-01-15 13:06:46 - INFO - ran-wd_query_format_response :: END - 8282
2025-01-15 13:06:46 - INFO - ran-main_wd_query_and_process :: END


In [12]:
resp['user_question'] = user_question
resp

{'s3_url': ['s3://dl-dish-wrls-whlsl-network-documents-cpni-p/gpl_documents/mavenir/Mavenir Parameter documentation/Mavenir Parameters v0.9.9/GPL v0.9.9.xlsx',
  's3://dl-dish-wrls-whlsl-network-documents-cpni-p/gpl_documents/mavenir/Mavenir Parameter documentation/Mavenir Parameters v0.9.8/Mavenir_Dish_GPL_v0.9.8_Aug_16.xlsx',
  's3://dl-dish-wrls-whlsl-network-documents-cpni-p/gpl_documents/mavenir/Mavenir Parameter documentation/Mavenir Parameters v0.9.7/Mavenir_Dish_GP_v0.9.7_Jun_21_2022.xlsx'],
 'confidence_score': [0.31171, 0.29861, 0.29492],
 'passages': 'Plus: 1) 5G Data Model and related lecturing record is added in sheet ‘Definition’. 2) Cell level  parameters  filtering method is added in sheet ‘Definition’. 3) Few  parameters  update per feedback from customer or Mav field team                        v0.9.4 2/3/22  GPL  refinement is made for  Dish  top 515  parameters , which can be filtered out by column V ‘Labels’ and label is ‘ Dish _Top_515_Subset’. In detail: 1) More 

In [13]:
print(resp['passages'])

Plus: 1) 5G Data Model and related lecturing record is added in sheet ‘Definition’. 2) Cell level  parameters  filtering method is added in sheet ‘Definition’. 3) Few  parameters  update per feedback from customer or Mav field team                        v0.9.4 2/3/22  GPL  refinement is made for  Dish  top 515  parameters , which can be filtered out by column V ‘Labels’ and label is ‘ Dish _Top_515_Subset’. In detail: 1) More words added in description field for better customer understanding. 2) For BW dependency  parameters , provide additional words in column F ‘Description’ and add label ‘BW_Specific_ Parameter ’ in column V ‘Labels’. 3) Correct Inaccurate Ranges / System Recommendation mismatch. 4) If restriction in the supported range, mention the same in column F ‘Description’. 5) For deprecated  parameters , add label ‘Deprecated_ Parameter ’  in column V ‘Labels’                        v0.9.5 3/11/22  GPL  is updated referring to 17.5 Mavenir RAN  Parameter  Audit (Slides by  

In [16]:

summary = await analyzer(resp)
print(summary)

2025-01-15 13:07:15 - INFO - user_question:None - RAN-Analyzer : START
2025-01-15 13:07:19 - INFO - user_question:None - RAN-Analyzer : END


I don't have a tool available to answer this question. However, I can tell you that the information provided in the document is related to the updates and changes made to the GPL (General Parameter List) for Dish/Boost, but it does not provide a direct answer to the question about the default value of the scellDeactInterval parameter.

If you have access to the GPL document or the related sheets ('Definition' or 'GPL'), you may be able to find the information you're looking for. Alternatively, you can try searching for the parameter in the provided document or checking the labels and descriptions in the 'Definition' sheet to see if it provides any clues about the default value.


#### Below is TESTING FOR FILTER

In [40]:
import requests
filter_vendor = "metadata.vendor::\"samsung\""
# user_query = "what is the Dish GPL Value for the parameter cbPreamblePerSSB ?"
# user_query = "How does Codec Adaption works in Mavenir? any difference with Samsung?"

# Define variables
url = "https://cpd-cpd-inst-operands.apps.watsonx.cpni-wl-dw-d.aws.dishcloud.io/discovery/cpd-inst-operands-wd/instances/1730206863041206/api/v2/projects/826cc215-0482-454f-b649-217416edb7e3/query?version=2022-08-01"
# wd_bearer_token = "your_bearer_token"  # Replace with your actual token
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {WD_BEARER_TOKEN}",
}
payload = {
    "count": 3,
    "return": ["title", "metadata"],
    "passages": {
        "enabled": True,
        "characters": 2000,
        "per_document": True,
        "find_answers": True,
        "fields": ["text"]
    },
    # "filter": filter_vendor,
    "natural_language_query":user_question,
    "table_results": {"enabled": False}
}

# Send the POST request
response = requests.post(url, headers=headers, json=payload, verify=False)

# Check response status and content
if response.status_code == 200:
    print("Response:", response.json())
else:
    print("Error:", response.status_code, response.text)

Error: 503 {
  "status_code" : 503,
  "errors" : [ {
    "code" : "query_unavailable",
    "message" : "Query timed out, please try again.",
    "more_info" : "https://cloud.ibm.com/docs/discovery-data"
  } ],
  "trace" : "f4cda93f-9d8a-4014-81c1-a1fbff71b539"
}


In [132]:
wd_res = await wd_query_entities(user_query, filter_params=None)

2025-01-01 18:22:41 - INFO - ran-wd_query_entities :: START
2025-01-01 18:22:44 - INFO - ran-wd_query_entities :: END


In [129]:
wd_res

{'matching_results': 569,
 'retrieval_details': {'document_retrieval_strategy': 'untrained'},
 'aggregations': [{'type': 'term',
   'field': 'enriched_text.entities.text',
   'name': 'entities',
   'results': [{'key': '5', 'matching_results': 513},
    {'key': '2', 'matching_results': 452},
    {'key': '1', 'matching_results': 451},
    {'key': '3', 'matching_results': 450},
    {'key': '4', 'matching_results': 443},
    {'key': '6', 'matching_results': 437},
    {'key': '8', 'matching_results': 434},
    {'key': '7', 'matching_results': 433},
    {'key': '10', 'matching_results': 427},
    {'key': '9', 'matching_results': 425}]}],
 'results': [{'document_id': '28477a62-f7c7-42fb-ab6e-a0abb4e42928',
   'result_metadata': {'collection_id': '8a59273c-3e71-5716-0000-019416333f91',
    'document_retrieval_source': 'search',
    'confidence': 0.06244},
   'metadata': {'file_name': '5232_Obsolete_Mavenir_5G_RAN_SA_FDD_Feature Description Guide.pdf',
    'release': '5232',
    's3_url': 's3:/

In [125]:
user_query

'How does Codec Adaption works in Mavenir? any difference with Samsung?'

In [120]:
resp_ = response.json()
resp_['results']

[{'document_id': '28477a62-f7c7-42fb-ab6e-a0abb4e42928',
  'result_metadata': {'collection_id': '8a59273c-3e71-5716-0000-019416333f91',
   'document_retrieval_source': 'search',
   'confidence': 0.06348},
  'metadata': {'file_name': '5232_Obsolete_Mavenir_5G_RAN_SA_FDD_Feature Description Guide.pdf',
   'release': '5232',
   's3_url': 's3://dl-dish-wrls-whlsl-network-documents-cpni-p/product_manuals/mavenir/5232_Obsolete_Mavenir_5G_RAN_SA_FDD_Feature Description Guide.pdf',
   'parent_document_id': '28477a62-f7c7-42fb-ab6e-a0abb4e42928',
   'file_last_modified': '2024-11-05 15:50:05+00:00',
   'xls_tab': 'NOT_DEFINED',
   'version': 'NOT_DEFINED',
   'file_category': 'feature',
   'bucket': 'dl-dish-wrls-whlsl-network-documents-cpni-p',
   'file_key': 'product_manuals/mavenir/5232_Obsolete_Mavenir_5G_RAN_SA_FDD_Feature Description Guide.pdf',
   'node': 'NOT_DEFINED',
   'parent_folder': 'product_manuals',
   'size': 10407841,
   'file_type': 'pdf',
   'vendor': 'mavenir'},
  'document

In [133]:

respp = await wd_query_format_response(wd_res)
# resp_
respp['user_question'] = user_query
summary = await analyzer(respp)
summary

2025-01-01 18:22:50 - INFO - ran-wd_query_format_response :: START
2025-01-01 18:22:50 - INFO - ran-wd_query_format_response :: END - 8518
2025-01-01 18:22:50 - INFO - user_question:what is the Dish GPL Value for the parameter cbPreamblePerSSB ? - RAN-Analyzer : START


{'user_question': 'what is the Dish GPL Value for the parameter cbPreamblePerSSB ?', 'passages': 'Revision Revision History/Notes Revision History/Notes *All  Dish   GPL   parameters  in the respective market/AOI for optimization activity need to be aligned as per  Dish   GPL  To include the new changes with description in this live  GPL  critical  parameters  document     *Changing  values  for mobility thresholds should follow general rule and not create a ping-pong scenario: A1 > A2, A2 >= A51…      *For any additional  parameter  trial sync up with NRF for a review with proposal      *" Dish _ GPL _ Parameters " tab has global  values  that mostly applicable to all sites, "Connected mobility" and "Idle_Mode" tabs are specific to markets based on available band/freq      Revision  Parameter  Notes Owner Action Date  - ns pCCHCfgInfo/ns being changed from ns_two to ns_one as part of CR fix to defect - DEF-6706  Dish  NRF Added 7/25/22 - anrNeighborDelete part of ANR enhancement reque

2025-01-01 18:22:52 - INFO - user_question:what is the Dish GPL Value for the parameter cbPreamblePerSSB ? - RAN-Analyzer : END


'The Dish GPL Value for the parameter cbPreamblePerSSB is 64.'

In [13]:
url = "http://127.0.0.1:8000/ran/watsonx/ran/ran_sql"

data = {
  "thread_id": "4",
  # "message": "how many incidents are raised in houston?"
  "message": "who is cm of karnataka?"
}



In [14]:
import requests
# url = 'http://127.0.0.1:8000/watsonx/rca/graph'
# data = {
#     'prompt': 'fetch rca for the above incidents',
#     "user_id":"abc",
#     "conversation_id" : "d1f8f55d-3488-4b9e-b894-ee8b0fee6fae"
# }
for chunk in requests.post(url, stream= True, json=data):
    print(chunk.decode('utf-8'), end="", flush = True)

The question is not related to the RAN documents that I have been trained on. Please provide a relevant query and I will be happy to help!

In [16]:
import csv
import psycopg2
from psycopg2 import sql, extensions
import os

# Define your database connection details from environment variables
db_config = {
    'dbname': 'tme_noc',
    'user': 'postgres',
    'password': 'yMu1RfMtLF5tNir9',
    'host': 'localhost',
    'port': '5432'
}

# Define the CSV file path (ensure the file path is correct)
csv_file_path = './mcms_cm_topology_state_cucp_12hr.csv'

# Define the table name (correct the typo in the table name)
table_name = 'ran.mcms_cm_topology_state_cucp_12hr'

# Max length for column names in PostgreSQL (63 characters)
MAX_COL_LENGTH = 63

# Max field size for a column (set as per your system's limitation, 1MB in this example)
MAX_FIELD_SIZE = 10**6  # 1MB (this can be increased)

# Function to clean and truncate column names
def clean_column_name(header):
    cleaned = header.replace('-', '_')  # Replace hyphens with underscores
    cleaned = cleaned.replace('.', '_')  # Replace periods with underscores
    if len(cleaned) > MAX_COL_LENGTH:
        cleaned = cleaned[:MAX_COL_LENGTH] + "_alias"  # Alias if too long
    return cleaned

# Function to truncate fields that exceed the maximum field size
def truncate_field(field):
    if isinstance(field, str) and len(field) > MAX_FIELD_SIZE:
        return field[:MAX_FIELD_SIZE]  # Truncate field to the maximum size
    return field

# Function to insert data dynamically
def insert_data(cursor, table_name, cleaned_headers, rows):
    placeholders = ', '.join(['%s'] * len(cleaned_headers))  # Create the correct number of placeholders
    insert_query = sql.SQL(f"INSERT INTO {table_name} ({', '.join([f'{header}' for header in cleaned_headers])}) VALUES ({placeholders})")
    cursor.executemany(insert_query, rows)  # Insert data in batches
    print(f"{len(rows)} rows inserted successfully.")

# Connect to PostgreSQL
try:
    # Establish connection and increase the field size limit
    conn = psycopg2.connect(**db_config)
    conn.set_client_encoding('UTF8')  # Ensure UTF8 encoding
    cursor = conn.cursor()

    # Increase the maximum field size limit in psycopg2
    extensions.register_type(extensions.UNICODE)  # Handle Unicode characters properly
    extensions.register_type(extensions.UNICODEARRAY)  # Handle arrays of Unicode characters

    print("Connected to the database.")

    # Open the CSV file
    with open(csv_file_path, mode='r', encoding='utf-8') as file:
        reader = csv.reader(file)
        headers = next(reader)  # Get the column names from the first row
        
        # Clean headers
        cleaned_headers = [clean_column_name(header) for header in headers]
        
        # Batch insert data row by row
        batch_size = 1000  # Adjust batch size as needed
        rows_to_insert = []
        
        for row in reader:
            # Truncate fields that are too large
            row = [truncate_field(field) for field in row]
            rows_to_insert.append(row)
            
            if len(rows_to_insert) >= batch_size:
                insert_data(cursor, table_name, cleaned_headers, rows_to_insert)
                rows_to_insert = []  # Reset the list for the next batch

        # Insert any remaining rows that didn't fill a full batch
        if rows_to_insert:
            insert_data(cursor, table_name, cleaned_headers, rows_to_insert)
        
        # Commit the transaction
        conn.commit()
        print("Transaction committed successfully.")

except psycopg2.DatabaseError as e:
    print(f"Database error occurred: {e}")
except Exception as e:
    print(f"Error: {e}")
finally:
    # Ensure the connection is properly closed
    try:
        cursor.close()
        conn.close()
        print("Connection closed.")
    except Exception as e:
        print(f"Error closing connection: {e}")


Connected to the database.
20 rows inserted successfully.
Transaction committed successfully.
Connection closed.


In [1]:
## sql rename

import os
import psycopg2
from typing import Dict
from langchain_ibm import WatsonxLLM
from dotenv import load_dotenv
import datetime
import traceback
from decimal import Decimal
from utils.logger import logger

try:
    pg_host = os.environ["POSTGRESS_URL"]
    pg_db = os.environ["POSTGRES_DB"]
    pg_user = os.environ["POSTGRESS_USERNAME"]
    pg_pass = os.environ["POSTGRESS_PASSWORD"]
    pg_port = os.environ["POSTGRES_PORT"] 

except Exception as e:
    print(e)
    load_dotenv()
    print("Loading Environmment Variables from local .env file")
    pg_host = os.environ["POSTGRESS_URL"]
    pg_db = os.environ["POSTGRES_DB"]
    pg_user = os.environ["POSTGRESS_USERNAME"]
    pg_pass = os.environ["POSTGRESS_PASSWORD"]
    pg_port = os.environ["POSTGRES_PORT"]

In [15]:
cursor

<cursor object at 0x17eaafb50; closed: 0>

In [37]:
column_mapping = {
    # "ne_id": "ne-id",
    "ne_type": "ne-type",
    # "du_reparenting": "du-reparenting",
    "system_type": "system-type",
    "user_label": "user-label",
    # "ne_id": "ne-id",
    # "du_reparenting": "du-reparenting",
    # "o_ran_radio_unit_o_ran_radio_unit_info_object": "o-ran-radio-unit.o-ran-radio-unit-info.object",
    # "o_ran_radio_unit_o_ran_radio_unit_info_o_ran_ru_id": "o-ran-radio-unit.o-ran-radio-unit-info.o-ran-ru-id",
    # "o_ran_radio_unit_o_ran_radio_unit_info_msr_operational_mode": "o-ran-radio-unit.o-ran-radio-unit-info.msr-operational-mode",
    # "o_ran_radio_unit_o_ran_radio_unit_info_operational_mode": "o-ran-radio-unit.o-ran-radio-unit-info.operational-mode",
    # "o_ran_radio_unit_o_ran_radio_unit_info_serial_number": "o-ran-radio-unit.o-ran-radio-unit-info.serial-number",
    # "o_ran_radio_unit_o_ran_radio_unit_info_sub_type": "o-ran-radio-unit.o-ran-radio-unit-info.sub-type",
    # "o_ran_radio_unit_o_ran_radio_unit_info_unit_type": "o-ran-radio-unit.o-ran-radio-unit-info.unit-type",
    # "o_ran_radio_unit_o_ran_radio_unit_info_user_label": "o-ran-radio-unit.o-ran-radio-unit-info.user-label",
    # "antenna_line_device_antenna_line_device_info_object": "antenna-line-device.antenna-line-device-info.object",
    # "antenna_line_device_antenna_line_device_info_antenna_line_devic": "antenna-line-device.antenna-line-device-info.antenna-line-device-id",
    # "antenna_line_device_antenna_line_device_info_antenna_serial_num": "antenna-line-device.antenna-line-device-info.antenna-serial-number",
    # "antenna_line_device_antenna_line_device_info_user_label": "antenna-line-device.antenna-line-device-info.user-label",
    # "antenna_line_device_antenna_line_device_info_vendor_code": "antenna-line-device.antenna-line-device-info.vendor-code",
    # "software_inventory_software_slot_object": "software-inventory.software-slot.object",
    # "software_inventory_software_slot_name": "software-inventory.software-slot.name",
    # "software_inventory_software_slot_access": "software-inventory.software-slot.access",
    # "software_inventory_software_slot_active": "software-inventory.software-slot.active",
    # "software_inventory_software_slot_build_id": "software-inventory.software-slot.build-id",
    # "software_inventory_software_slot_build_name": "software-inventory.software-slot.build-name",
    # "software_inventory_software_slot_build_version": "software-inventory.software-slot.build-version",
    # "software_inventory_software_slot_product_code": "software-inventory.software-slot.product-code",
    # "software_inventory_software_slot_running": "software-inventory.software-slot.running",
    # "software_inventory_software_slot_status": "software-inventory.software-slot.status",
    # "software_inventory_software_slot_vendor_code": "software-inventory.software-slot.vendor-code",
    # "ret_ret_info_object": "ret.ret-info.object",
    # "ret_ret_info_antenna_id": "ret.ret-info.antenna-id",
    # "ret_ret_info_antenna_model_number": "ret.ret-info.antenna-model-number",
    # "ret_ret_info_antenna_operating_band": "ret.ret-info.antenna-operating-band",
    # "ret_ret_info_antenna_serial_number": "ret.ret-info.antenna-serial-number",
    # "ret_ret_info_beam_width": "ret.ret-info.beam-width",
    # "ret_ret_info_config_antenna_bearing": "ret.ret-info.config-antenna-bearing",
    # "ret_ret_info_config_base_station_id": "ret.ret-info.config-base-station-id",
    # "ret_ret_info_config_install_date": "ret.ret-info.config-install-date",
    # "ret_ret_info_config_installed_tilt": "ret.ret-info.config-installed-tilt",
    # "ret_ret_info_config_installer_id": "ret.ret-info.config-installer-id",
    # "ret_ret_info_config_sector_id": "ret.ret-info.config-sector-id",
    # "ret_ret_info_config_tilt": "ret.ret-info.config-tilt",
    # "ret_ret_info_current_antenna_bearing": "ret.ret-info.current-antenna-bearing",
    # "ret_ret_info_current_base_station_id": "ret.ret-info.current-base-station-id",
    # "ret_ret_info_current_install_date": "ret.ret-info.current-install-date",
    # "ret_ret_info_current_installed_tilt": "ret.ret-info.current-installed-tilt",
    # "ret_ret_info_current_installer_id": "ret.ret-info.current-installer-id",
    # "ret_ret_info_current_sector_id": "ret.ret-info.current-sector-id",
    # "ret_ret_info_current_tilt": "ret.ret-info.current-tilt",
    # "ret_ret_info_gain": "ret.ret-info.gain",
    # "ret_ret_info_maximum_tilt": "ret.ret-info.maximum-tilt",
    # "ret_ret_info_minimum_tilt": "ret.ret-info.minimum-tilt",
    # "ret_ret_info_user_label": "ret.ret-info.user-label"
}

In [38]:
your_table = "dl_silver_ran_samsung_piiprod.usm_cm_ret_state_1d"
conn = psycopg2.connect(
    dbname=pg_db,
    user=pg_user,
    password=pg_pass,
    host=pg_host,
    port=pg_port
)
cursor = conn.cursor()

# Generate and execute SQL rename commands
for old_col, new_col in column_mapping.items():
    sql = f'ALTER TABLE {your_table} RENAME COLUMN {old_col} TO "{new_col}"'
    print(sql)
    cursor.execute(sql)
    

# Commit changes and close connection
conn.commit()
cursor.close()
conn.close()

print("Columns renamed successfully!")

ALTER TABLE dl_silver_ran_samsung_piiprod.usm_cm_ret_state_1d RENAME COLUMN ne_type TO "ne-type"
ALTER TABLE dl_silver_ran_samsung_piiprod.usm_cm_ret_state_1d RENAME COLUMN system_type TO "system-type"
ALTER TABLE dl_silver_ran_samsung_piiprod.usm_cm_ret_state_1d RENAME COLUMN user_label TO "user-label"
Columns renamed successfully!


In [43]:
from fastapi import APIRouter, HTTPException, status
from pydantic import BaseModel, Field
from typing import Optional, Dict
from utils.logger import logger
from llms.llms import chatmodel_ran_automation
from langchain_core.prompts import ChatPromptTemplate
import traceback
# import utils.ran_automation_validation as validate
# import utils.ret_transaction_logging as execute_params
# import utils.user_authorization_wrapper as validate_email
# import utils.ret_create_csv_file as csv_util
# import utils.ret_transaction_logging as log_transaction
import time
# Define API Router
ran_automation_router = APIRouter(prefix="/watsonx/ran/automation", tags=["RAN Automation"])

# Response Model
class ExtractParamsResponse(BaseModel):
    is_valid: bool
    query_type: Optional[str] = None
    table_type: Optional[str] = None
    params: Optional[Dict[str, Optional[str]]] = None
    message: str
    error_code: Optional[str] = None

class ExtractParamsRequest(BaseModel):
    query: str
    transaction_id: str

# Request Parsing Models
class TiltModel(BaseModel):
    vendor: Optional[str] = Field(None, description="Name of the vendor, e.g., Samsung or Mavenir")
    tilt_value: Optional[str] = Field(None, description="tilt value which needs to be updated in given in input it must be a number, if not present then None")
    cell_id: Optional[str] = Field(None, description="Cell name or ID provided in the input")
    hldc_add: Optional[str] = Field(None, description=" HLDC (High-Level Data Link Control) addresses provided in the input")

# Request Parsing Models
class GNBModel(BaseModel):
    vendor: Optional[str] = Field(None, description="Name of the vendor, e.g., Samsung or Mavenir")
    gnb : Optional[str] = Field(None, description="the gnb provided in the input")
    cellid: Optional[str] = Field(None, description="Cell name or ID provided in the input")
    threshold_rsrp : Optional[str] = Field(None, description="threshold_rsrp provided in the input")
    hysteresis : Optional[str] = Field(None, description="hysteresis provided in the input")
    time_to_trigger : Optional[str] = Field(None, description="time_to_trigger provided in the input")
    index : Optional[str] = Field(None, description="index provided in the input")

In [56]:
# import traceback
import os
# from utils.logger import logger
# from langchain.prompts import PromptTemplate
# from prompts.prompts import (QUERY_CLASSIFIER_PROMPT_TEMPLATE_LLAMA,
#                             ANALYZER_PROMPT_TEMPLATE_LLAMA)
# from langchain_ibm import WatsonxLLM
from langchain_ibm import ChatWatsonx
from dotenv import load_dotenv
# import ast
# import json
# from utils.llm_utils import call_with_timeout
import json
from langchain_core.prompts import PromptTemplate

In [23]:
try:
    # Fetch environment variables
    WATSONX_URL = os.environ["WATSONX_URL"]
    WATSONX_APIKEY = os.environ["WATSONX_API_KEY"]
    WATSONX_PROJECT_ID = os.environ["WATSONX_PROJECT_ID"]
    WATSONX_MODEL_ID_LLAMA = os.environ["WATSONX_MODEL_ID_LLAMA"]
    LLM_LLAMA_PARAMS = os.environ["LLM_LLAMA_PARAMS"] 
    TIMEOUT_DURATION_LLM_ANALYZER = int(os.environ["TIMEOUT_DURATION_LLM_ANALYZER"])
    TIMEOUT_DURATION_LLM = int(os.environ["TIMEOUT_DURATION_LLM"])
    LLM_LLAMA_PARAMS_CHAT = os.environ["LLM_LLAMA_PARAMS_CHAT"]
    WATSONX_MODEL_ID_MISTRAL_LARGE = os.environ["WATSONX_MODEL_ID_MISTRAL_LARGE"]
    LLM_MISTRAL_PARAMS_CHAT = os.environ["LLM_MISTRAL_PARAMS_CHAT"]
    WATSONX_MODEL_ID_LLAMA_3 = os.environ["WATSONX_MODEL_ID_LLAMA_3"]
    WATSONX_MODEL_ID_AUTOMATION_PARAMS = os.environ["WATSONX_MODEL_ID_AUTOMATION_PARAMS"]

except Exception as e:
    print(e)
    print("Loading Environmment Variables from local .env file")
    load_dotenv()
    WATSONX_URL = os.environ["WATSONX_URL"]
    WATSONX_APIKEY = os.environ["WATSONX_API_KEY"]
    WATSONX_PROJECT_ID = os.environ["WATSONX_PROJECT_ID"]
    WATSONX_MODEL_ID_LLAMA = os.environ["WATSONX_MODEL_ID_LLAMA"]
    LLM_LLAMA_PARAMS = os.environ["LLM_LLAMA_PARAMS"]
    TIMEOUT_DURATION_LLM_ANALYZER = int(os.environ["TIMEOUT_DURATION_LLM_ANALYZER"])
    TIMEOUT_DURATION_LLM = int(os.environ["TIMEOUT_DURATION_LLM"])
    LLM_LLAMA_PARAMS_CHAT = os.environ["LLM_LLAMA_PARAMS_CHAT"]
    WATSONX_MODEL_ID_MISTRAL_LARGE = os.environ["WATSONX_MODEL_ID_MISTRAL_LARGE"]
    LLM_MISTRAL_PARAMS_CHAT = os.environ["LLM_MISTRAL_PARAMS_CHAT"]
    WATSONX_MODEL_ID_LLAMA_3 = os.environ["WATSONX_MODEL_ID_LLAMA_3"]
    WATSONX_MODEL_ID_AUTOMATION_PARAMS = os.environ["WATSONX_MODEL_ID_AUTOMATION_PARAMS"]

In [35]:
WATSONX_MODEL_ID_AUTOMATION_PARAMS='{"decoding_method": "greedy", "max_new_tokens": 300, "min_new_tokens": 0, "temperature": 0, "stop_sequences":[]}'
chatmodel_ran_automation = ChatWatsonx(
    model_id='meta-llama/llama-3-3-70b-instruct',
    project_id=WATSONX_PROJECT_ID,
    url=WATSONX_URL,
    apikey=WATSONX_APIKEY,
    params=json.loads(WATSONX_MODEL_ID_AUTOMATION_PARAMS),
    streaming=False,
)

In [ ]:
# Request Parsing Models
class GNBModel(BaseModel):
    vendor: Optional[str] = Field(None, description="Name of the vendor, e.g., Samsung or Mavenir")
    gnb : Optional[str] = Field(None, description="the gnb provided in the input")
    cellid: Optional[str] = Field(None, description="Cell name or ID provided in the input")
    threshold_rsrp : Optional[str] = Field(None, description="threshold_rsrp provided in the input")
    hysteresis : Optional[str] = Field(None, description="hysteresis provided in the input")
    time_to_trigger : Optional[str] = Field(None, description="time-to-trigger provided in the input")
    index : Optional[str] = Field(None, description="index provided in the input")

In [54]:
async def extract_gnb_params(query: str):
    """
    Extracts gnb-related parameters like vendor, gnodeb_id, cell_id, threshold_rsrp, hysteresis, time_to_trigger and index from the given input.
    """
    prompt = PromptTemplate.from_template(
    # prompt = ChatPromptTemplate.from_template(
        "Extract the following parameters from the input query:\n"
        "1. Vendor: Name of the vendor, e.g., Samsung, Mavenir.\n"
        "2. gnb: The specified gnb will be used to update the other values.\n"
        "3. cellid: The name or identifier of the cell.\n"
        "4. threshold_rsrp: The specified a5-threshold1-rsrp value to be updated.\n"
        "-- example : paramter name will be a5-threshold1-rsrp and value can be an integer value 20,50...etc"
        "5. hysteresis: The specified hysteresis value to be updated.\n"
        "6. time_to_trigger: The specified time-to-trigger value to be updated.\n"
        "7. index: The name or indentifier of the index.\n"
        "**Respond with JSON only. Do not include explanations or extra text.**\n"
        "**Example response:**\n"
        '{ "vendor": "Samsung", "gnb": "731025", "cellid": "10", "threshold_rsrp": "50", "hysteresis": null, "time_to_trigger": null, "index": "2" }'
        "\n\nInput query: {query}"
    )
    chain = prompt | chatmodel_ran_automation #.with_structured_output(GNBModel)
    response = await chain.ainvoke({"query": query})
    return response

In [57]:
query =  "Change a5-threshold1-rsrp for cell id 10, reportConfig index 2 to value 50 for gnb 731025 for samsung"
# query = "Change hysteris for cellid 12  to value 2  for gnb 731025 for eventA3" 
params = await extract_gnb_params(query)
params

KeyError: 'Input to PromptTemplate is missing variables {\' "vendor"\'}.  Expected: [\' "vendor"\', \'query\'] Received: [\'query\']\nNote: if you intended { "vendor"} to be part of the string and not a variable, please escape it with double curly braces like: \'{{ "vendor"}}\'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT '

In [61]:
gnb_prompt = """Extract the following parameters from the input query and return them in strict JSON format:
    
    1. vendor: Name of the vendor (e.g., Samsung, Mavenir)
    2. gnb: The specified gNB
    3. cellid: Cell ID
    4. threshold_rsrp: a5-threshold1-rsrp value
    5. hysteresis: hysteresis value (if mentioned)
    6. time_to_trigger: time-to-trigger value (if mentioned)
    7. index: Report config index (if mentioned)

    **Respond with JSON only. Do not include explanations or extra text.**
    Example:
    user_question : Change a5-threshold1-rsrp for cell id 10, reportConfig index 2 to value 50 for gnb 731025 for samsung
    **response:**
    ```json
    {{ 
        "vendor": "Samsung", 
        "gnb": "731025", 
        "cellid": "10", 
        "threshold_rsrp": "50", 
        "hysteresis": null, 
        "time_to_trigger": null, 
        "index": "2" 
    }}
    ```

    Input query: {query}
    """

In [64]:
class GNBModel(BaseModel):
    vendor: str = Field(..., description="Name of the vendor, e.g., Samsung or Mavenir")
    gnb: str = Field(..., description="The gNB provided in the input")
    cellid: str = Field(..., description="Cell name or ID provided in the input")
    threshold_rsrp: str = Field(..., description="a5-threshold1-rsrp value in the input")
    hysteresis: str = Field(None, description="Hysteresis value (if provided)")
    time_to_trigger: str = Field(None, description="Time-to-trigger value (if provided)")
    index: str = Field(None, description="ReportConfig index (if provided)")

In [68]:
from langchain_core.exceptions import OutputParserException
async def extract_gnb_params(query: str) -> GNBModel:
    """
    Extracts gnb-related parameters like vendor, gnodeb_id, cell_id, threshold_rsrp, hysteresis, time_to_trigger and index from the given input.
    """
    prompt = ChatPromptTemplate.from_template(gnb_prompt)
    chain = prompt | chatmodel_ran_automation.with_structured_output(GNBModel)
    try:
        response = await chain.ainvoke({"query": query})
        return response
    except OutputParserException as e:
        print("Parsing Error:", e)
        return None  # Return None or handle gracefully

In [73]:
# query =  "Change a5-threshold1-rsrp for cell id 10, reportConfig index 2 to value 50 for gnb 731025 for samsung"
# query = "Change hysteris for cellid 12  to value 2  for gnb 731025 for eventA3" 
# query = "Change a5-threshold1-rsrp for cellid 6 , reportConfig index 2 to value 50 for gnb 731025"
# query = "Change time-to-trigger for cellid 12  to value ms640 for gnb 731025 for eventA2 "
query = "Change hysteris for cellid 12  to value 2  for gnb 731025 for eventA3 index is 5678"
params = await extract_gnb_params(query)
params

GNBModel(vendor='null', gnb='731025', cellid='12', threshold_rsrp='null', hysteresis='2', time_to_trigger='null', index='5678')